# Data Cleaning




## Objective

The objective of this stage is to improve data quality and consistency across the Airbnb datasets for Berlin and Bangkok. This includes identifying and addressing missing values, duplicates, formatting inconsistencies, and invalid records to ensure the data is reliable and suitable for exploratory analysis and dashboard development.


In [28]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).



## Data Quality Issues Identified

* Missing values are present in both numerical fields (such as price and reviews per month) and categorical fields (such as host name and license). These can affect statistical calculations and category-based analysis, so each affected column must be handled carefully during data cleaning.

* Price-related fields require validation to ensure values are numeric and within realistic ranges. Extremely high, zero, or incorrectly formatted prices can distort average price calculations and lead to misleading comparisons between cities.

* There may be duplicate listings based on listing identifiers, which can inflate property counts and review statistics. Verifying and removing duplicates is necessary to ensure each property is represented only once.

* Text-based fields like neighbourhood and room type may contain inconsistent formatting (such as extra spaces or case differences), which can split the same category into multiple groups and affect grouping and visualization accuracy.

* Review-related fields contain missing or sparse values, often indicating new or inactive listings rather than data errors. These should be interpreted carefully to avoid removing meaningful information about market activity.


In [29]:
import pandas as pd
BASE_PATH = "/content/drive/MyDrive/AlmaBetter/Module_4/data"

berlin_listings = pd.read_csv(f"{BASE_PATH}/berlin/listings.csv")
berlin_reviews = pd.read_csv(f"{BASE_PATH}/berlin/reviews.csv")
berlin_neighbourhoods = pd.read_csv(f"{BASE_PATH}/berlin/neighbourhoods.csv")

bangkok_listings = pd.read_csv(f"{BASE_PATH}/bangkok/listings.csv")
bangkok_reviews = pd.read_csv(f"{BASE_PATH}/bangkok/reviews.csv")
bangkok_neighbourhoods = pd.read_csv(f"{BASE_PATH}/bangkok/neighbourhoods.csv")

# Create working copies to preserve raw data
berlin_listings_clean = berlin_listings.copy()
berlin_reviews_clean = berlin_reviews.copy()
berlin_neighbourhoods_clean = berlin_neighbourhoods.copy()

bangkok_listings_clean = bangkok_listings.copy()
bangkok_reviews_clean = bangkok_reviews.copy()
bangkok_neighbourhoods_clean = bangkok_neighbourhoods.copy()


## Cleaning Strategy

* Apply the same cleaning methods to both Berlin and Bangkok datasets to ensure fair and reliable comparison.

* Keep the original raw data unchanged and perform all cleaning on working copies to preserve data integrity.

* Treat missing values based on their business importance and impact on analysis rather than applying a single rule to all fields.

* Identify and remove duplicate listings using unique listing identifiers to avoid inflated counts and biased statistics.

* Standardize categorical text fields by correcting case and spacing to ensure consistent grouping and accurate visualizations.

* Validate numerical columns to confirm correct formats and realistic value ranges before performing statistical analysis.


## Handling Missing Values

* Missing values will be handled based on the importance of each column for analysis and visualization.

* Critical fields such as price, room type, and neighbourhood will be reviewed carefully, as they directly impact insights and dashboards.
For less critical fields (for example, review-related or optional host details), missing values may be retained or handled selectively.

* No missing values are modified at this stage; decisions will be applied in subsequent cleaning steps.



In [30]:
# Drop records where price is missing, as price is critical for analysis
berlin_listings_clean = berlin_listings_clean.dropna(subset=["price"])
bangkok_listings_clean = bangkok_listings_clean.dropna(subset=["price"])


## Handling Duplicates

Duplicate records will be checked using unique listing identifiers.
If duplicate entries are found, only one valid record will be retained to avoid double counting in analysis.
The focus will be on ensuring that each listing is represented only once in the cleaned dataset.
No duplicate removal is performed at this stage; this section only defines the approach.



In [31]:
# Remove duplicate listings based on listing id
berlin_listings_clean = berlin_listings_clean.drop_duplicates(subset=["id"])
bangkok_listings_clean = bangkok_listings_clean.drop_duplicates(subset=["id"])



## Standardizing Formats

* Text-based fields such as neighbourhood names and room types will be standardized to ensure consistency across records.

* This includes fixing letter casing, trimming extra spaces, and resolving minor naming variations.

* Price and date fields will be reviewed to ensure they follow a consistent format suitable for analysis and visualization.

* At this stage, this section only documents the standardization approach; no transformations are applied yet.




In [32]:
# Standardize text fields: trim spaces and convert to lowercase
columns_to_clean = ["neighbourhood", "room_type"]
dataframes_listings = {"Berlin_listings": berlin_listings_clean, "Bangkok_listings": bangkok_listings_clean}
dataframes_neighbourhoods = {"Berlin_neighbourhoods":berlin_neighbourhoods_clean, "Bangkok_neighbourhoods": bangkok_neighbourhoods_clean}

for name, df in dataframes_listings.items():
    for col in columns_to_clean:
        df[col] = df[col].astype(str).str.strip().str.lower()
        print(f"\n{name}")
        print(f"\n{col}")
        print(df[col].unique()[:20])
for name, df in dataframes_neighbourhoods.items():
    df["neighbourhood"] = df["neighbourhood"].astype(str).str.strip().str.lower()
    print(f"\n{name}")
    print(df['neighbourhood'].unique()[:20])



Berlin_listings

neighbourhood
['prenzlauer berg südwest' 'prenzlauer berg nordwest' 'reuterstraße'
 'brunnenstr. süd' 'tempelhofer vorstadt' 'helmholtzplatz'
 'düsseldorfer straße' 'schöneberg-nord' 'südliche luisenstadt'
 'frankfurter allee süd fk' 'neue kantstraße' 'prenzlauer berg süd'
 'brunnenstr. nord' 'prenzlauer berg nord' 'schmargendorf'
 'alexanderplatz' 'blankenfelde/niederschönhausen' 'schöneberg-süd'
 'südliche friedrichstadt' 'wiesbadener straße']

Berlin_listings

room_type
['entire home/apt' 'private room' 'shared room' 'hotel room']

Bangkok_listings

neighbourhood
['ratchathewi' 'don mueang' 'rat burana' 'bang rak' 'khlong toei'
 'bang kapi' 'lat krabang' 'vadhana' 'bang sue' 'phra khanong'
 'chatu chak' 'sathon' 'khlong san' 'yan na wa' 'bang kho laen' 'bang na'
 'din daeng' 'thawi watthana' 'parthum wan' 'bangkok yai']

Bangkok_listings

room_type
['entire home/apt' 'private room' 'shared room' 'hotel room']

Berlin_neighbourhoods
['barstraße' 'charlottenburg nord

In [40]:
# Remove records where invalid price values present

for name, df in dataframes_listings.items():
    df_clean = df[df["price"].notna()]   # keep only rows with valid price
    dataframes_listings[name] = df_clean # put cleaned df back into dictionary

# Also update original variables
berlin_listings_clean = dataframes_listings["Berlin_listings"]
bangkok_listings_clean = dataframes_listings["Bangkok_listings"]
# Check for any Nan present
print(berlin_listings_clean.isna().sum())
print(bangkok_listings_clean.isna().sum())




id                                   0
name                                 0
host_id                              0
host_name                            2
neighbourhood_group                  0
neighbourhood                        0
latitude                             0
longitude                            0
room_type                            0
price                                0
minimum_nights                       0
number_of_reviews                    0
last_review                       2070
reviews_per_month                 2070
calculated_host_listings_count       0
availability_365                     0
number_of_reviews_ltm                0
license                           1909
dtype: int64
id                                    0
name                                  0
host_id                               0
host_name                             7
neighbourhood_group               23273
neighbourhood                         0
latitude                              0
longi

## Output Datasets

* The output of the data cleaning process will be cleaned and standardized datasets for both Berlin and Bangkok.

* These datasets will be used as inputs for exploratory data analysis and dashboard creation.

* Raw datasets will remain unchanged to preserve original data integrity.

* The cleaned datasets will be generated and validated in the execution phase of data cleaning.


In [42]:
import os

# Define the full paths for the output directories
output_berlin_dir = os.path.join(BASE_PATH, "berlin")
output_bangkok_dir = os.path.join(BASE_PATH, "bangkok")

# Create the directories if they don't exist
os.makedirs(output_berlin_dir, exist_ok=True)
os.makedirs(output_bangkok_dir, exist_ok=True)

# Save cleaned datasets for downstream analysis
berlin_listings_clean.to_csv(os.path.join(output_berlin_dir, "listings_clean.csv"), index=False)
bangkok_listings_clean.to_csv(os.path.join(output_bangkok_dir, "listings_clean.csv"), index=False)

berlin_reviews_clean.to_csv(os.path.join(output_berlin_dir, "reviews_clean.csv"), index=False)
bangkok_reviews_clean.to_csv(os.path.join(output_bangkok_dir, "reviews_clean.csv"), index=False)

berlin_neighbourhoods_clean.to_csv(os.path.join(output_berlin_dir, "neighbourhoods_clean.csv"), index=False)
bangkok_neighbourhoods_clean.to_csv(os.path.join(output_bangkok_dir, "neighbourhoods_clean.csv"), index=False)